<a href="https://colab.research.google.com/github/JanXanevo/PrototypeRecommendations/blob/main/06_snowflakeImportOffline_01_12_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snowflake-sqlalchemy

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [ ]:
from google.colab import drive
import os


# Warengruppen und PNKs aus dem Gartenbereich laden

drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/BAHAG/Data')

df2 = pd.read_excel('/content/drive/My Drive/BAHAG/Data/Kopie von Garten_WGR_PNK.xlsx')

In [ ]:
GartenPNr =df2['VARIANT_PRODUCT'].tolist()
GartenPNr = [value for value in GartenPNr if isinstance(value, (int, float))]


In [ ]:
# snowflake login connection
SNOWFLAKE_USER = 'GUGELM'
SNOWFLAKE_PASSWORD = 'pGtYSXfdP9hJTbP'
SNOWFLAKE_ACC_ID = 'xt02912.europe-west4.gcp'
SNOWFLAKE_DB = 'DC_BI_DB'
SNOWFLAKE_SCHEMA = 'RPT_PRODUCTCREATION'
SNOWFLAKE_ROLE = 'ROLE_PT_PRODUCTCREATION'
SNOWFLAKE_WH = 'COMPUTE_WH'

# Snowflake-Verbindungszeichenfolge erstellen
connection_string = f'snowflake://{SNOWFLAKE_USER}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACC_ID}/{SNOWFLAKE_DB}/{SNOWFLAKE_SCHEMA}?role={SNOWFLAKE_ROLE}&warehouse={SNOWFLAKE_WH}'

# SQLAlchemy-Engine erstellen
engine = create_engine(connection_string)

# Teste die Verbindung
connection = engine.connect()

# Teile die Liste in Gruppen von 1000 Werten auf
group_size = 1000
grouped_values = [GartenPNr[i:i + group_size] for i in range(0, len(GartenPNr), group_size)]

all_results = []

# Führe separate Abfragen für jede Gruppe durch
for group in grouped_values:
    placeholders = ','.join([':%s' % i for i in range(len(group))])

    sql_query = f"""
    SELECT PRODUKTNR, VERKAUFSBELEG_DATUM, MENGE
    FROM "DC_BI_DB"."RPT_PRODUCTCREATION"."BASKETS_RECEIPTS_STATIONARY_V"
    WHERE MENGE > 0
    AND TRY_CAST(PRODUKTNR AS NUMBER) IS NOT NULL
    AND PRODUKTNR IN ({placeholders})
    QUALIFY VERKAUFSBELEG_DATUM = LAG(VERKAUFSBELEG_DATUM) OVER (ORDER BY VERKAUFSBELEG_DATUM)
    OR VERKAUFSBELEG_DATUM = LEAD(VERKAUFSBELEG_DATUM) OVER (ORDER BY VERKAUFSBELEG_DATUM)
    ORDER BY VERKAUFSBELEG_DATUM
    """

    result = connection.execute(sqlalchemy.text(sql_query), **{str(i): val for i, val in enumerate(group)})

    # Füge die Ergebnisse dieser Abfrage zur Liste hinzu
    all_results.append(pd.DataFrame(result.fetchall(), columns=result.keys()))

# Verbinde alle DataFrames in der Liste zu einem einzigen DataFrame
final_dataframe = pd.concat(all_results, ignore_index=True)

# Nun ist das finale DataFrame mit allen Ergebnissen verfügbar
print(final_dataframe)

In [ ]:
drive.mount('/content/drive')

df_path ='/content/drive/My Drive/BAHAG/Data/offlineProducts.csv'
final_dataframe.to_csv(df_path, index=False)